In [9]:
# this will be used to access the bigquery client
from google.cloud import bigquery
# this will be used to access the service account modules - passwords and access tokens
from google.oauth2 import service_account
# this will be used for returning data, increases the speed
from google.cloud import bigquery_storage
# for os library
import os


#general dataframe usage
import pandas as pd
# required for certain returns
import pandas_gbq
# exception for a short script
from google.api_core.exceptions import NotFound

In [10]:
# Path to service account if stored locally
SERVICE_ACCOUNT_JSON = r"..\portfolio2026-485323-70c4d609b156.json"

# Credentials to access Big Query via the service account
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_JSON)

# BigQuery client
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

# BigQuery Storage client
bq_storage_client = bigquery_storage.BigQueryReadClient(credentials=credentials)

# Gather the datasets inside the project for validation of location

In [11]:
datasets = list(client.list_datasets())

In [12]:
if datasets:
    print("Datasets in project:")
    for dataset in datasets:
        print(dataset.dataset_id)
else:
    print("No datasets found.")


# function to see if table already exists in big query data set
def table_exists(client, full_table_id):
    try:
        client.get_table(full_table_id)
        return True
    except NotFound:
        return False

Datasets in project:
portfolio1


# Read in files and concatenate into a single data frame

In [13]:
dataframes = []

row_count = 0

for dirpath, dirs, files in os.walk(r"data_files"):
    for n, file in enumerate(files):
        df = pd.read_csv(os.path.join(dirpath,file))
        df = df.drop('Player-additional', axis=1)
        df['Year'] = 2020 + n + 1
        if 'MP▼' in df.columns:
            df.rename(columns={'MP▼':'MP'}, inplace=True)

        if 'Team' in df.columns:
            df.rename(columns={'Team':'Tm'}, inplace=True) 
        dataframes.append(df)
        row_count += df.shape[0]

nba_df = pd.concat(dataframes)

assert nba_df.shape[0] == row_count, f"Expected {row_count} rows, but got {nba_df.shape[0]}"

In [14]:
nba_df.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,TRB,AST,STL,BLK,TOV,PF,PTS,Year,Trp-Dbl,Awards
0,1,Precious Achiuwa,PF,21,MIA,61,4,737,124,228,...,208,29,20,28,43,91,304,2021,NaN,NaN
1,2,Jaylen Adams,PG,24,MIL,7,0,18,1,8,...,3,2,0,0,0,1,2,2021,NaN,NaN
2,3,Steven Adams,C,27,NOP,58,58,1605,189,308,...,514,111,54,38,78,113,438,2021,NaN,NaN
3,4,Bam Adebayo,C,23,MIA,64,64,2143,456,800,...,573,346,75,66,169,145,1197,2021,NaN,NaN
4,5,LaMarcus Aldridge,C,35,TOT,26,23,674,140,296,...,118,49,11,29,27,47,352,2021,NaN,NaN


In [15]:
nba_df[nba_df['Player'] == "D'Angelo Russell"]

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,TRB,AST,STL,BLK,TOV,PF,PTS,Year,Trp-Dbl,Awards
576,443,D'Angelo Russell,PG,24,MIN,42,26,1196,281,652,...,111,244,45,18,112,68,796,2021,NaN,NaN
645,486,D'Angelo Russell,PG,25,MIN,65,65,2077,401,975,...,216,460,62,22,165,130,1176,2022,NaN,NaN
543,426,D'Angelo Russell,PG,26,TOT,71,71,2304,445,948,...,215,437,70,29,186,140,1263,2023,NaN,NaN
544,426,D'Angelo Russell,PG,26,MIN,54,54,1778,338,727,...,165,334,60,21,147,117,967,2023,NaN,NaN
545,426,D'Angelo Russell,PG,26,LAL,17,17,526,107,221,...,50,103,10,8,39,23,296,2023,NaN,NaN
173,143,D'Angelo Russell,PG,28,2TM,58,36,1480,249,638,...,162,297,56,23,108,115,732,2024,0.0,NaN
174,143,D'Angelo Russell,PG,28,LAL,29,10,763,129,311,...,80,136,23,4,50,53,359,2024,0.0,NaN
175,143,D'Angelo Russell,PG,28,BRK,29,26,717,120,327,...,82,161,33,19,58,62,373,2024,0.0,NaN


# Process for Big Query upload

In [16]:
nba_df.columns = nba_df.columns.str.lower().str.strip().str.replace(r"[^\w]+","_", regex=True)

In [17]:
# Write to bigquery

# Write truncate will overwrite the table instead of appending
job_config = bigquery.LoadJobConfig(write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
                                    autodetect=True)

full_table_id = f"{credentials.project_id}.portfolio1.nba_data"

if table_exists(client, full_table_id):
    print(f"Table {full_table_id} already exists.")
    print("Overwriting existing table...")
else:
    print("Creating new table...")

load_job = client.load_table_from_dataframe(nba_df, full_table_id, job_config=job_config)
load_job.result()

Table portfolio2026-485323.portfolio1.nba_data already exists.
Overwriting existing table...


LoadJob<project=portfolio2026-485323, location=US, id=9fc5f937-92ae-4fec-99c7-8ddb9b1e33b9>